In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [2]:
batch_size= 32
learning_rate = 0.0002
epoch = 100

In [3]:
transform = transforms.Compose([transforms.CenterCrop(224),transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5),(0.5,0.5,0.5))])

In [4]:
trainset = dset.CIFAR10('./',train =True, download = True, transform = transform)
testset = dset.CIFAR10('./',train =False, download = True, transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_loader = DataLoader(trainset,batch_size=batch_size,shuffle=True,num_workers=2)
test_loader = DataLoader(testset,batch_size=batch_size,shuffle=False,num_workers=2)

In [6]:
class block(nn.Module):
    
    def __init__(self,in_dim,mid_dim,out_dim):
        super(block,self).__init__()
        self.res =  nn.Sequential(
            nn.Conv2d(in_dim,mid_dim,3,1,1),
            nn.ReLU(),
            nn.Conv2d(mid_dim,out_dim,3,1,1)
            )
               
    def forward(self,x):
        out = self.res(x)
        #print("block res :",out.size())
        #out = torch.cat([out,x],dim =1) 
        out = torch.add(out,x)
        #print("block res+ x :",out.size())
        out = Variable(out)
    
        return out


In [7]:
class model(nn.Module):
    
    def __init__(self):
        super(model,self).__init__()
        self.layer_1 = nn.Sequential(
            nn.Conv2d(3,32,7,2,3),
            nn.ReLU(),
            nn.MaxPool2d(3,2,1),
        )
        self.block1 = block(32,64,32)
        self.block2 = block(32,64,32)
        self.fc1 =nn.Linear(100352,1000)
        self.fc2 =nn.Linear(1000,10)
    def forward(self,y):
        y = self.layer_1(y)
        #print(y.size())
        y = self.block1(y)
        #print(y.size())
        y = self.block2(y)
        #print(y.size())
        
        y = y.view(y.size(0),-1)
        y = self.fc1(y)
        y = self.fc2(y)
        return y

resnet = model()

In [8]:
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(),lr =0.0002 )

In [9]:
    for j,(img, label) in enumerate(train_loader):
        img = Variable(img)
        label = Variable(label)
        optimizer.zero_grad()
        
        output = resnet(img)
        print(output.size())
        print(label.size())
        loss = loss_f(output,label)
        loss.backward()
        optimizer.step()
        print(loss)

torch.Size([32, 10])
torch.Size([32])
tensor(2.3755, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(56.1840, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(48.0927, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(49.0618, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(59.7707, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(46.1574, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(53.1658, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(61.5964, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(62.0798, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(40.4856, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(49.0406, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(57.7520, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Si

tensor(7.9450, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(11.9031, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(8.4281, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(7.1136, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(5.9740, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(7.2662, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(6.3279, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(6.0762, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(5.9226, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(9.6419, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(8.5284, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(7.6760, grad_fn=<NllLossBackward>)
torch.Size([32, 10])
torch.Size([32])
tensor(10.5366, grad_fn=<NllLossBackwar

KeyboardInterrupt: 

In [125]:
correct = 0
total = 0

for data in test_loader:
    data = img, label
    img = Variable(img)
    label = Variable(label)
    
    output = resnet(img)
    _,pridicted = torch.max(output,-1)
    total += label.size(0)
    correct += (pridicted == label).sum()
    
print(' accuracy: %f'%(100*correct/total))
    

 accuracy: 43.000000


In [126]:
class block1(nn.Module):
    
    def __init__(self,in_dim,mid_dim,out_dim):
        super(block1,self).__init__()
        self.res =  nn.Sequential(
            nn.Conv2d(in_dim,mid_dim,3,1,1),
            nn.ReLU(),
            nn.Conv2d(mid_dim,out_dim,3,1,1)
            )
               
    def forward(self,x):
        out = self.res(x)
        #print("block res :",out.size())
        #out = torch.cat([out,x],dim =1) 
        #out = torch.add(out,x)
        #print("block res+ x :",out.size())
        #out = Variable(out)
    
        return out


In [127]:
class model1(nn.Module):
    
    def __init__(self):
        super(model1,self).__init__()
        self.layer_1 = nn.Sequential(
            nn.Conv2d(3,32,7,2,3),
            nn.ReLU(),
            nn.MaxPool2d(3,2,1),
        )
        self.block1 = block1(32,64,32)
        self.block2 = block1(32,64,32)
        self.fc1 =nn.Linear(100352,1000)
        self.fc2 =nn.Linear(1000,10)
    def forward(self,y):
        y = self.layer_1(y)
        #print(y.size())
        y = self.block1(y)
        #print(y.size())
        y = self.block2(y)
        #print(y.size())
        
        y = y.view(y.size(0),-1)
        y = self.fc1(y)
        y = self.fc2(y)
        return y

resnet_noadd = model1()

In [128]:
    for j,(img, label) in enumerate(test_loader):
        img = Variable(img)
        label = Variable(label)
        optimizer.zero_grad()
    
        output = resnet_noadd(img)
        loss = loss_f(output,label)
        loss.backward()
        optimizer.step()
   
        print(loss)

tensor(2.3066, grad_fn=<NllLossBackward>)
tensor(2.3102, grad_fn=<NllLossBackward>)
tensor(2.3020, grad_fn=<NllLossBackward>)
tensor(2.3002, grad_fn=<NllLossBackward>)
tensor(2.3063, grad_fn=<NllLossBackward>)
tensor(2.3050, grad_fn=<NllLossBackward>)
tensor(2.2950, grad_fn=<NllLossBackward>)
tensor(2.3038, grad_fn=<NllLossBackward>)
tensor(2.3019, grad_fn=<NllLossBackward>)
tensor(2.3055, grad_fn=<NllLossBackward>)
tensor(2.3100, grad_fn=<NllLossBackward>)
tensor(2.2979, grad_fn=<NllLossBackward>)
tensor(2.3015, grad_fn=<NllLossBackward>)
tensor(2.3083, grad_fn=<NllLossBackward>)
tensor(2.3076, grad_fn=<NllLossBackward>)
tensor(2.2998, grad_fn=<NllLossBackward>)
tensor(2.3074, grad_fn=<NllLossBackward>)
tensor(2.3139, grad_fn=<NllLossBackward>)
tensor(2.3038, grad_fn=<NllLossBackward>)
tensor(2.3041, grad_fn=<NllLossBackward>)
tensor(2.3097, grad_fn=<NllLossBackward>)
tensor(2.3081, grad_fn=<NllLossBackward>)
tensor(2.2957, grad_fn=<NllLossBackward>)
tensor(2.3027, grad_fn=<NllLossBac

tensor(2.3046, grad_fn=<NllLossBackward>)
tensor(2.3026, grad_fn=<NllLossBackward>)
tensor(2.2953, grad_fn=<NllLossBackward>)
tensor(2.3112, grad_fn=<NllLossBackward>)
tensor(2.2896, grad_fn=<NllLossBackward>)
tensor(2.2980, grad_fn=<NllLossBackward>)
tensor(2.2917, grad_fn=<NllLossBackward>)
tensor(2.3048, grad_fn=<NllLossBackward>)
tensor(2.3048, grad_fn=<NllLossBackward>)
tensor(2.3064, grad_fn=<NllLossBackward>)
tensor(2.2953, grad_fn=<NllLossBackward>)
tensor(2.2974, grad_fn=<NllLossBackward>)
tensor(2.3074, grad_fn=<NllLossBackward>)
tensor(2.3029, grad_fn=<NllLossBackward>)
tensor(2.3003, grad_fn=<NllLossBackward>)
tensor(2.3015, grad_fn=<NllLossBackward>)
tensor(2.3067, grad_fn=<NllLossBackward>)
tensor(2.2982, grad_fn=<NllLossBackward>)
tensor(2.3075, grad_fn=<NllLossBackward>)
tensor(2.3109, grad_fn=<NllLossBackward>)
tensor(2.3068, grad_fn=<NllLossBackward>)
tensor(2.3039, grad_fn=<NllLossBackward>)
tensor(2.2994, grad_fn=<NllLossBackward>)
tensor(2.3001, grad_fn=<NllLossBac

In [131]:
correct = 0
total = 0

for data in train_loader:
    data = img,label
    img = Variable(img)
    label = Variable(label)
    
    output = resnet_noadd(img)
    _,pridicted = torch.max(output,-1)
    total += label.size(0)
    correct += (pridicted == label).sum()
    
print(' accuracy: %f'%(100*correct/total))
    

 accuracy: 18.000000


In [111]:
a = torch.ones(5,3)
print(a)
print(a.size(),a.size(0), a.size(1), a.size(-1), a.size(-2))
print(a.data) # a 랑 같음 
print(a.shape) # a.size() 랑 같음
#print(a.shape(0)) - 안불러짐

a = a.numpy() # numpy 로 변경 
# print(a.data()) 출력안됨

print(a.data)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
torch.Size([5, 3]) 5 3 3 5
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
torch.Size([5, 3])


TypeError: 'torch.Size' object is not callable